In [7]:
import MetaTrader5 as mt5
from config import *

In [8]:
result = mt5.initialize()
if login_number == mt5.account_info().login and result is True:
    print("Connection to MetaTrader5 established")
else:
    print("Failed to connect at account #{}, error code: {}".format(login_number, mt5.last_error()))
    mt5.shutdown()

Connection to MetaTrader5 established


In [ ]:
#BUY Market Order Parameters

volume = 0.1
action = mt5.TRADE_ACTION_DEAL
order_type = mt5.ORDER_TYPE_BUY

stop_loss = 122964.0
take_profit = 125298.0

def get_market_price(symbol, order_type):
    symbol_info = mt5.symbol_info(symbol)
    
    if symbol_info is None:
        raise ValueError(f"Failed to get tick for {symbol}: MT5 Error: {mt5.last_error()}")
    elif order_type == 0:
        return symbol_info.ask
    elif order_type == 1:
        return symbol_info.bid
    else:
        raise ValueError("order_type must be 0 (buy) or 1 (sell)")
 
#Buy Order Request   
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": "BTCUSD",
    "volume": volume,
    "type": mt5.ORDER_TYPE_BUY,
    "price": get_market_price("BTCUSD", 0),         #0 is to BUY, 1 is to SELL
    "sl": stop_loss,
    "tp": take_profit,
    "deviation": 20,
    "magic": 1,
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,
}

res = mt5.order_send(request)
res 

OrderSendResult(retcode=10009, deal=36838679, order=41616850, volume=0.1, price=124106.0, bid=124088.0, ask=124106.0, comment='Request executed', request_id=1274242759, retcode_external=0, request=TradeRequest(action=1, magic=1, order=0, symbol='BTCUSD', volume=0.1, price=124111.0, stoplimit=0.0, sl=122964.0, tp=125298.0, deviation=20, type=0, type_filling=1, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))

In [28]:
#Modify TP & SL

#Set these values to 0.0 if you don't want TP or SL on your trade
stop_loss = 121864.0
take_profit = 125898.0

symbol = 'BTCUSD'    #Adjust the name to fit the exact commodity you're looking for

positions = mt5.positions_get()
print(f"Open Positions: {positions} \n")


for position in positions:
    if position.symbol == symbol:
        print(position.ticket)
        pos = positions.index(position)     #We use index operator because positions is a Tuple
        print(pos, symbol)
        
        request = {
            'action': mt5.TRADE_ACTION_SLTP,
            'position': position.ticket,
            'sl': stop_loss,
            'tp': take_profit
        }
        
        res = mt5.order_send(request)
        print(res)
    else: 
        print(f"There is no trade for: {symbol}")


Open Positions: (TradePosition(ticket=41615893, time=1759722629, time_msc=1759722629066, time_update=1759722629, time_update_msc=1759722629066, type=0, magic=1, identifier=41615893, reason=3, volume=0.01, price_open=123548.0, sl=121864.0, tp=125898.0, price_current=124045.0, swap=0.0, profit=4.24, symbol='BTCUSD', comment='python script op', external_id=''),) 

41615893
0 BTCUSD
OrderSendResult(retcode=10025, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='No changes', request_id=0, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='', volume=0.0, price=0.0, stoplimit=0.0, sl=121864.0, tp=125898.0, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=41615893, position_by=0))


In [44]:
#CLOSE POSITION

symbol = 'BTCUSD'    #Adjust the name to fit the exact commodity you're looking for

positions = mt5.positions_get()
print(f"Open Positions: {positions} \n")


def get_close_price(symbol, order_type):
    symbol_info = mt5.symbol_info(symbol)
    
    if symbol_info is None:
        raise ValueError(f"Failed to get tick for {symbol}: MT5 Error: {mt5.last_error()}")
    elif order_type == 0:
        return symbol_info.bid
    elif order_type == 1:
        return symbol_info.ask
    else:
        raise ValueError("order_type must be 0 (buy) or 1 (sell)")    
    
    
def reverse_type(order_type):
    #This closes the trade position by reversing the order type
    if order_type == mt5.ORDER_TYPE_BUY:
        return mt5.ORDER_TYPE_SELL
    if order_type == mt5.ORDER_TYPE_SELL:
        return mt5.ORDER_TYPE_BUY


for position in positions:
    if position.symbol == symbol:
        print(position.ticket)
        pos = positions.index(position)     #We use index operator because positions is a Tuple
        print(f"Position: {pos},", symbol)
        
        request = {
            'action': mt5.TRADE_ACTION_DEAL,
            'position': position.ticket,
            'symbol': symbol,
            'volume': position.volume,
            "type": reverse_type(position.type),
            "price": get_close_price(symbol, position.type),
            'deviation': 20,
            'magic': 0,
            "comment": "python script open",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        
        res = mt5.order_send(request)
        print(res)
    else: 
        print(f"There is no trade for: {symbol}")

Open Positions: (TradePosition(ticket=41616850, time=1759726815, time_msc=1759726815126, time_update=1759726866, time_update_msc=1759726866170, type=0, magic=1, identifier=41616850, reason=3, volume=0.05, price_open=124106.0, sl=122964.0, tp=125298.0, price_current=123953.0, swap=0.0, profit=-6.52, symbol='BTCUSD', comment='python script op', external_id=''), TradePosition(ticket=41617020, time=1759727166, time_msc=1759727166472, time_update=1759727166, time_update_msc=1759727166472, type=0, magic=1, identifier=41617020, reason=3, volume=0.1, price_open=123998.0, sl=122964.0, tp=125298.0, price_current=123953.0, swap=0.0, profit=-3.84, symbol='BTCUSD', comment='python script op', external_id='')) 

41616850
Position: 0, BTCUSD
OrderSendResult(retcode=10009, deal=36838860, order=41617033, volume=0.05, price=123953.0, bid=123953.0, ask=123971.0, comment='Request executed', request_id=1274242765, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='BTCUSD', volume=

In [40]:
#PARTIAL CLOSE POSITION

perc_close = 0.5

symbol = 'BTCUSD'    #Adjust the name to fit the exact commodity you're looking for

positions = mt5.positions_get()
print(f"Open Positions: {positions} \n")


def reverse_type(order_type):
    #This closes the trade position by reversing the order type
    if order_type == mt5.ORDER_TYPE_BUY:
        return mt5.ORDER_TYPE_SELL
    if order_type == mt5.ORDER_TYPE_SELL:
        return mt5.ORDER_TYPE_BUY

def get_close_price(symbol, order_type):
    symbol_info = mt5.symbol_info(symbol)
    
    if symbol_info is None:
        raise ValueError(f"Failed to get tick for {symbol}: MT5 Error: {mt5.last_error()}")
    elif order_type == 0:
        return symbol_info.bid
    elif order_type == 1:
        return symbol_info.ask
    else:
        raise ValueError("order_type must be 0 (buy) or 1 (sell)")    
    

for position in positions:
    if position.symbol == symbol:
        print(position.ticket)
        pos = positions.index(position)     #We use index operator because positions is a Tuple
        print(f"Position: {pos},", symbol)
        
        request = {
            'action': mt5.TRADE_ACTION_DEAL,
            'position': position.ticket,
            'symbol': symbol,
            'volume': round(position.volume * perc_close, 2),   #Close Half of the Position
            "type": reverse_type(position.type),
            "price": get_close_price(symbol, position.type),
            'deviation': 20,
            'magic': 0,
            "comment": "python script open",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        
        res = mt5.order_send(request)
        print(res)
    else: 
        print(f"There is no trade for: {symbol}")

Open Positions: (TradePosition(ticket=41616850, time=1759726815, time_msc=1759726815126, time_update=1759726815, time_update_msc=1759726815126, type=0, magic=1, identifier=41616850, reason=3, volume=0.1, price_open=124106.0, sl=122964.0, tp=125298.0, price_current=124134.0, swap=0.0, profit=2.39, symbol='BTCUSD', comment='python script op', external_id=''),) 

41616850
Position: 0, BTCUSD
OrderSendResult(retcode=10009, deal=36838724, order=41616894, volume=0.05, price=124139.0, bid=124139.0, ask=124157.0, comment='Request executed', request_id=1274242761, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='BTCUSD', volume=0.05, price=124134.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=1, type_time=0, expiration=0, comment='python script open', position=41616850, position_by=0))


In [42]:
#BUY LIMIT ORDER Parameters

volume = 0.1
action = mt5.TRADE_ACTION_PENDING
order_type = mt5.ORDER_TYPE_BUY_LIMIT

stop_loss = 122964.0
take_profit = 125298.0
limit_price = 124000.0

#Buy Order Request   
request = {
    "action": action,
    "symbol": "BTCUSD",
    "volume": volume,
    "type": order_type,
    "price": limit_price,         #0 is to BUY, 1 is to SELL
    "sl": stop_loss,
    "tp": take_profit,
    "deviation": 20,
    "magic": 1,
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,
}

res = mt5.order_send(request)
res 

OrderSendResult(retcode=10009, deal=0, order=41617020, volume=0.1, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=1274242762, retcode_external=0, request=TradeRequest(action=5, magic=1, order=0, symbol='BTCUSD', volume=0.1, price=124000.0, stoplimit=0.0, sl=122964.0, tp=125298.0, deviation=20, type=2, type_filling=1, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))

In [ ]:
#SELL STOP ORDER Parameters

volume = 0.1
action = mt5.TRADE_ACTION_PENDING
order_type = mt5.ORDER_TYPE_SELL_LIMIT

stop_loss = 122964.0
take_profit = 125298.0
limit_price = 124000.0

#Buy Order Request   
request = {
    "action": action,
    "symbol": "BTCUSD",
    "volume": volume,
    "type": order_type,
    "price": limit_price,         #0 is to BUY, 1 is to SELL
    "sl": stop_loss,
    "tp": take_profit,
    "deviation": 20,
    "magic": 1,
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,
}

res = mt5.order_send(request)
res 

In [ ]:
#Modify Pending Order

symbol = 'BTCUSD'    #Adjust the name to fit the exact commodity you're looking for
price = 125000.0

positions = mt5.positions_get()

action = mt5.TRADE_ACTION_MODIFY


for position in positions:
    if position.symbol == symbol:
        print(position.ticket)
        pos = positions.index(position)     #We use index operator because positions is a Tuple
        print(f"Position: {pos},", symbol, position.order)
        
        request = {
            'action': action,
            'order': position.order,
            'price': price,
            'sl': stop_loss,
            'tp': take_profit,
        }
    else: 
        print(f"There is no trade for: {symbol}")

res = mt5.order_send(request)
res


In [ ]:
#Remove Pending Order

pending_orders = mt5.orders_get()
print('pending orders: ', pending_orders)

symbol = 'BTCUSD'

for position in positions:
    if position.symbol == symbol:
        print(position.ticket)
        pos = positions.index(position)     #We use index operator because positions is a Tuple
        print(f"Position: {pos},", symbol, position.order)
        
        request = {
            'action': mt5.TRADE_ACTION_REMOVE,
            'order': position.ticket
        }
    else: 
        print(f"There is no trade for: {symbol}")